In [1]:
%load_ext autoreload
%autoreload 2
from AmbientesMP import *
from random import choice
from busqueda import Nodo , nodo_hijo

## Back tracking

In [2]:
def back_tracking(prob, state):
    
    
    if prob.test_objetivo(state):
        return [state]
    
    for action in prob.acciones_aplicables(state):
        
        new_state = prob.transicion(estado = state,
                                    accion = action)
        
        result = back_tracking(prob, new_state)
        
        if result != False:
            return  [new_state] + result
        
    
    return False

## Bread first search

In [3]:
def bread_first_search(prob, state):
    
    frontier = [state]
    
    if prob.test_objetivo(state):
        return state
    
    while len(frontier) > 0:
        
        state = frontier.pop(0)
        
        for action in prob.acciones_aplicables(state):
            child = prob.transicion(state, action)
            
            if prob.test_objetivo(child):
                return child
            
            frontier.append(child)
            
    return False

## Depth limited search

In [4]:
def get_codes(node: Nodo):
    
    if node.madre == None:
        return [node.codigo]
    
    return  [] + [node.codigo] + get_codes(node.madre)

In [6]:
def is_cycle(node: Nodo):
    
    return len( set(get_codes(node)) ) != len(get_codes(node))

In [7]:
def expand(problema, node):
    
    state = node.estado
    nodes = []
    
    for action in problema.acciones_aplicables(state):
        child = nodo_hijo(problema, node, action)
        nodes.append(child)
        
    return nodes
    

In [8]:
def depth_(node):
    
    if node.madre == None:
        return 0
    
    return 1 + depth_(node.madre)   

In [9]:
def depth_limited_search(prob, l):
    
    node = Nodo(estado = prob.estado_inicial,
                madre  = None,
                accion = None, 
                costo_camino = 0,
                codigo = 'origin')
    
    frontier  = [node]
    result = "falla"
    counter = "0"
    
    while len(frontier) > 0:
        
        node = frontier.pop()
        
        if prob.test_objetivo(node.estado):
            return node
        
        if depth_(node) >= l:
            return "cutoff"
        
        elif not is_cycle(node):
            
            for child in expand(prob, node):
                frontier.append(child)
                
    return result

## best friend :D

In [10]:
import numpy as np

class ListaPrioritaria():
    
    def __init__(self):
        self.diccionario = {}
        
    def __str__(self):
        cadena = '['
        inicial = True
        for costo in self.diccionario:
            elementos = self.diccionario[costo]
            for elemento in elementos:
                if inicial:
                    cadena += '(' + str(elemento) + ',' + str(costo) + ')'
                    inicial = False
                else:
                    cadena += ', (' + str(elemento) + ',' + str(costo) + ')'

        return cadena + ']'
    
    def push(self, elemento, costo):
        try:
            self.diccionario[costo].append(elemento)
        except:
            self.diccionario[costo] = [elemento]
            
    def pop(self):
        min_costo  = min(list(self.diccionario.keys()))
        candidatos = self.diccionario[min_costo]
        elemento   = candidatos.pop()
        
        if len(candidatos) == 0:
            del self.diccionario[min_costo]
        return elemento
    
    def is_empty(self):
        return len(self.diccionario) == 0
        

In [11]:
def costs(self, state, action):
    
    """
        INPUT:
                action: tupla de la forma ('S', 'aSb'), por ejemplo.
                
        OUTPUT:
                costo de tomar la acción (dependerá de que tan largo 
                sea el string, y esto lo hacemos para incentivar
                que acabe rápido)
    """

    return len(action[1])

In [12]:
from types import MethodType

def best_first_search(problem, fun):
    
    #asigna la función "fun" como función de costo
    problem.costo = MethodType(fun, problem)
    
    state = problem.estado_inicial
    code  = problem.codigo(s)
    node  = Nodo(estado = state,
                 madre  = None,
                 accion = None, 
                 costo_camino = 0,
                 codigo = 'origin')
    
    # pongo primer elemento en la frontera
    frontier = ListaPrioritaria()
    frontier.push(elemento = node,
                  costo    = 0)
    
    #explored dict
    explored = {code: 0}
    
    while not frontier.is_empty():
        
        node = frontier.pop()
        
        if problem.test_objetivo(node.estado):
            return node
    
        for child in expand(problem, node):
            state = child.estado
            code  = problem.codigo(state)
            cost  = child.costo_camino
            
            if (code not in explored.keys()) or (cost < explored[code]):
                frontier.push(child, cost)
                explored[code]  = cost
                
    return "falla"

## Greedy friend :C

In [13]:
def error_function(state: str, obj: str) -> int:
    
    """
        Input:
            state: String de la forma "aaSbb" o "aabb" por ejemplo.
            
        Output:
            nuestra loca euristica (int)
    """
    
    lenght_distance = ( (len(obj) - len(state))**2 )
    
    if len(state) > len(obj):
        char_distance   = len(state)**2
    else:
        char_distance   = sum([ 1 for l1, l2 in zip(list(obj), list(state))  if l1 != l2])**2

    return lenght_distance + char_distance

In [13]:
def greedy_search(problema, f):
    
    
    # INCLUYA AQUÍ SU CÓDIGO
    state = problema.estado_inicial
    node  = Nodo(estado = state, 
                 madre  = None, 
                 accion = None,
                 costo_camino = 0, 
                 codigo = "orgin")
    
    cost = f(problema.codigo(state), problema.w)
    
    frontier = ListaPrioritaria()
    frontier.push(node, cost)
    
    code = problema.codigo(state)
    explored = [code]
    
    while not frontier.is_empty():
        node = frontier.pop()

        if problema.test_objetivo(node.estado):
            return node
        
        for child in expand(problema, node):
            state = child.estado
            code  = problema.codigo(state)
            
            if code not in explored:
                cost  = f(problema.codigo(state), problema.w)
                frontier.push(child, cost)
                explored.append(code)
    
    return None
    
    # AQUÍ TERMINA SU CÓDIGO

## Pruebitas

In [14]:
regla1 = ('S', 'aSb')
regla2 = ('S', 'AB')
regla3 = ('A', 'a')
regla4 = ('B', 'b')
gramatica = [regla1, regla2, regla3, regla4]

w = "aaaaaaaaaaaaaaabbbbbbbbbbbbbbb"
prob = Parser(gramatica, w)
s = prob.estado_inicial

In [ ]:
"""
Note que si ponemos una palabra no satisfacible por la gramática, entonces no podremos 
solucionar el problema y se va a aquedar eternamente buscando solución.

Luego debemos darle la condición de parada, si |codigo| > |w| --> paila.
"""

In [15]:
ans = best_first_search(problem = prob, fun = costs)

In [16]:
prob.pintar_estado(ans.estado)

                                                         S                                                                 
  _______________________________________________________|___                                                               
 |   |                                                       S                                                             
 |   |    ___________________________________________________|___                                                           
 |   |   |   |                                                   S                                                         
 |   |   |   |    _______________________________________________|___                                                       
 |   |   |   |   |   |                                               S                                                     
 |   |   |   |   |   |    ___________________________________________|___                                                   
 |  

In [17]:
ans = depth_limited_search(prob = prob, l = len(w))

In [18]:
prob.pintar_estado(ans.estado)

                                                         S                                                                 
  _______________________________________________________|___                                                               
 |   |                                                       S                                                             
 |   |    ___________________________________________________|___                                                           
 |   |   |   |                                                   S                                                         
 |   |   |   |    _______________________________________________|___                                                       
 |   |   |   |   |   |                                               S                                                     
 |   |   |   |   |   |    ___________________________________________|___                                                   
 |  

In [19]:
ans = bread_first_search(prob  = prob, 
                         state = s)

In [20]:
prob.pintar_estado(ans)

                                                         S                                                                 
  _______________________________________________________|___                                                               
 |   |                                                       S                                                             
 |   |    ___________________________________________________|___                                                           
 |   |   |   |                                                   S                                                         
 |   |   |   |    _______________________________________________|___                                                       
 |   |   |   |   |   |                                               S                                                     
 |   |   |   |   |   |    ___________________________________________|___                                                   
 |  

In [21]:
ans = greedy_search(prob, error_function)

In [22]:
prob.pintar_estado(ans.estado)

                                                         S                                                                 
  _______________________________________________________|___                                                               
 |   |                                                       S                                                             
 |   |    ___________________________________________________|___                                                           
 |   |   |   |                                                   S                                                         
 |   |   |   |    _______________________________________________|___                                                       
 |   |   |   |   |   |                                               S                                                     
 |   |   |   |   |   |    ___________________________________________|___                                                   
 |  